<a href="https://colab.research.google.com/github/VeraMendes/DS-Unit-2-Kaggle-Challenge/blob/master/module3/Vera_Mendes_Assignment_kaggle_challenge_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science, Unit 2: Predictive Modeling

# Kaggle Challenge, Module 3


## Assignment
- [ ] [Review requirements for your portfolio project](https://lambdaschool.github.io/ds/unit2), then submit your dataset.
- [ ] Continue to participate in our Kaggle challenge. 
- [ ] Use scikit-learn for hyperparameter optimization with RandomizedSearchCV.
- [ ] Submit your predictions to our Kaggle competition. (Go to our Kaggle InClass competition webpage. Use the blue **Submit Predictions** button to upload your CSV file. Or you can use the Kaggle API to submit your predictions.)
- [ ] Commit your notebook to your fork of the GitHub repo.

## Stretch Goals

### Reading
- Jake VanderPlas, [Python Data Science Handbook, Chapter 5.3](https://jakevdp.github.io/PythonDataScienceHandbook/05.03-hyperparameters-and-model-validation.html), Hyperparameters and Model Validation
- Jake VanderPlas, [Statistics for Hackers](https://speakerdeck.com/jakevdp/statistics-for-hackers?slide=107)
- Ron Zacharski, [A Programmer's Guide to Data Mining, Chapter 5](http://guidetodatamining.com/chapter5/), 10-fold cross validation
- Sebastian Raschka, [A Basic Pipeline and Grid Search Setup](https://github.com/rasbt/python-machine-learning-book/blob/master/code/bonus/svm_iris_pipeline_and_gridsearch.ipynb)
- Peter Worcester, [A Comparison of Grid Search and Randomized Search Using Scikit Learn](https://blog.usejournal.com/a-comparison-of-grid-search-and-randomized-search-using-scikit-learn-29823179bc85)

### Doing
- Add your own stretch goals!
- Try other [categorical encodings](https://contrib.scikit-learn.org/categorical-encoding/). See the previous assignment notebook for details.
- In additon to `RandomizedSearchCV`, scikit-learn has [`GridSearchCV`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html). Another library called scikit-optimize has [`BayesSearchCV`](https://scikit-optimize.github.io/notebooks/sklearn-gridsearchcv-replacement.html). Experiment with these alternatives.
- _[Introduction to Machine Learning with Python](http://shop.oreilly.com/product/0636920030515.do)_ discusses options for "Grid-Searching Which Model To Use" in Chapter 6:

> You can even go further in combining GridSearchCV and Pipeline: it is also possible to search over the actual steps being performed in the pipeline (say whether to use StandardScaler or MinMaxScaler). This leads to an even bigger search space and should be considered carefully. Trying all possible solutions is usually not a viable machine learning strategy. However, here is an example comparing a RandomForestClassifier and an SVC ...

The example is shown in [the accompanying notebook](https://github.com/amueller/introduction_to_ml_with_python/blob/master/06-algorithm-chains-and-pipelines.ipynb), code cells 35-37. Could you apply this concept to your own pipelines?


### BONUS: Stacking!

Here's some code you can use to "stack" multiple submissions, which is another form of ensembling:

```python
import pandas as pd

# Filenames of your submissions you want to ensemble
files = ['submission-01.csv', 'submission-02.csv', 'submission-03.csv']

target = 'status_group'
submissions = (pd.read_csv(file)[[target]] for file in files)
ensemble = pd.concat(submissions, axis='columns')
majority_vote = ensemble.mode(axis='columns')[0]

sample_submission = pd.read_csv('sample_submission.csv')
submission = sample_submission.copy()
submission[target] = majority_vote
submission.to_csv('my-ultimate-ensemble-submission.csv', index=False)
```

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import plotly.express as px
import pandas_profiling
from sklearn.feature_selection import f_regression, SelectKBest
import itertools
import matplotlib.pyplot as plt
import seaborn as sns
import category_encoders as ce
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.compose import ColumnTransformer
from ipywidgets import interact
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn import metrics

In [0]:
# Merge train_features.csv & train_labels.csv
train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))

# Read test_features.csv & sample_submission.csv
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

In [0]:
def data_wrangle(X):
  # using same method for all data

  # create a copy
  X = X.copy()

  # values of latitude near zero will be treated as zero
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  
  # date_recorded appears as object, change into datetime format
  X['date_recorded'] = pd.to_datetime(X['date_recorded'],infer_datetime_format=True)  

  # extract year and month of inspection
  X['month_inspection'] = X['date_recorded'].dt.month
  X['year_inspection'] = X['date_recorded'].dt.year

  # extract number of years till inspection
  X['inspection_waiting_time'] = X['year_inspection'] - X['construction_year']

  # When columns have zeros and shouldn't, they are like null values.
  # So we will replace the zeros with nulls, and impute missing values later.
  cols_to_clean = ['longitude', 'latitude', 'amount_tsh','construction_year',
                   'gps_height', 'permit','public_meeting', 'region_code',
                   'source', 'quantity','quality_group', 'payment',
                   'water_quality', 'source_class','management',
                   'management_group', 'population','installer','funder',
                   'scheme_management', 'scheme_name']
  for col in cols_to_clean:
    X[col] = X[col].replace(0, np.nan)
    X[col] = X[col].replace('unknown', np.nan)
    X[col+'_MISSING'] = X[col].isnull()

  X['inspection_waiting_time_MISSING'] = X['inspection_waiting_time'].isnull()

  # binning numeric values
  cy_cut_points = [0, 0.1, 1980, 2000, 2013]
  cy_label_names = ['0-0.1', '1960-1980', '1981-2000', '2001-2013']
  X['construction_year_cat'] = pd.cut(X['construction_year'], cy_cut_points, labels=cy_label_names)

  gps_cut_points = [-63, 250, 1100, 1600, 2770]
  gps_label_names = ['-63-250', '251-1100', '1101-1600', '1601-2700']
  X['gps_height_cat'] = pd.cut(X['gps_height'], gps_cut_points, labels=gps_label_names)

  # quantity & quantity_group same as payment & payment_type are duplicates
  # drop duplicates, date_recorded, less relevance columns and columns used for binning
  X = X.drop(columns=['quantity_group','payment_type','date_recorded', 'recorded_by',
                      'gps_height','construction_year'])

  # create a top number of values in categorical features with too many unique or little values:
  ET_top6 = X['extraction_type'].value_counts()[:6].index
  ETG_top6 = X['extraction_type_group'].value_counts()[:6].index
  M_top7 = X['management'].value_counts()[:7].index
  S_top5 = X['source'].value_counts()[:5].index
  WQ_top2 = X['water_quality'].value_counts()[:2].index
  W_top4 = X['waterpoint_type'].value_counts()[:4].index


  # Where categorical value is not on the top values,
  # replace the value with 'Other'
  X.loc[~X['extraction_type'].isin(ET_top6), 'extraction_type'] = 'Other'
  X.loc[~X['extraction_type_group'].isin(ETG_top6), 'extraction_type_group'] = 'Other'
  X.loc[~X['management'].isin(M_top7), 'management'] = 'Other'
  X.loc[~X['source'].isin(S_top5), 'source'] = 'Other'
  X.loc[~X['water_quality'].isin(WQ_top2), 'water_quality'] = 'Other'
  X.loc[~X['waterpoint_type'].isin(W_top4), 'waterpoint_type'] = 'Other'

  # return my data_wrangle
  return X

train = data_wrangle(train)
test = data_wrangle(test) 


# defining my y vector
target = 'status_group'

# Get my train DF without the target
train_features = train.drop(columns=[target])

# list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

# arranging my X features and y vector
X_train = train[features]
y_train = train[target]
X_test = test[features]

In [0]:
pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators=100,max_depth=21, random_state=42,n_jobs=-1)
)

In [111]:
%%time
# using cross validation for my model
k = 5
scores = cross_val_score(pipeline, X_train, y_train, cv=k, n_jobs=-1, 
                         scoring='accuracy')
print(f'accuracy for {k} folds:', scores)

accuracy for 5 folds: [0.80927531 0.80481441 0.81001684 0.80488215 0.80459673]
CPU times: user 599 ms, sys: 72.7 ms, total: 671 ms
Wall time: 42.9 s


In [106]:
print('Model Hyperparameters:')
print(pipeline.named_steps['randomforestclassifier'])

Model Hyperparameters:
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)


In [115]:
# checking best hyperparameters with RandomizedSearchCV
param_distributions = {
    'simpleimputer__strategy': ['median', 'most_frequent'],
    'randomforestclassifier__n_estimators': randint(30, 300), 
    'randomforestclassifier__max_depth': [5, 10, 15, 22, None], 
    'randomforestclassifier__max_features': uniform(0, 1),

}

search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=10, 
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   10.6s
[Parallel(n_jobs=-1)]: Done   4 tasks      | elapsed:   56.3s
[Parallel(n_jobs=-1)]: Done   9 tasks      | elapsed:  2.0min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning:

A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.

[Parallel(n_jobs=-1)]: Done  14 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done  21 tasks      | elapsed:  6.4min
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  8.4min finished


In [116]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)

Best hyperparameters {'randomforestclassifier__max_depth': None, 'randomforestclassifier__max_features': 0.26189138044806115, 'randomforestclassifier__n_estimators': 286, 'simpleimputer__strategy': 'most_frequent'}
Cross-validation accuracy 0.8007575757575758


In [118]:
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

,3,4,8,5,2,0,9,7,1,6
mean_fit_time,57.9518,31.4587,23.5406,53.4675,19.8891,5.77867,21.4306,26.4542,39.2766,11.4083
std_fit_time,1.90305,1.48268,0.426253,1.84619,1.30003,0.0180052,6.0467,0.546665,1.19919,0.214855
mean_score_time,2.93584,1.56497,0.769671,1.41869,0.972571,1.06296,0.611259,1.06025,1.89484,0.952297
std_score_time,0.185821,0.108219,0.0200742,0.0421237,0.0775627,0.049116,0.173048,0.0774082,0.0584092,0.135591
param_randomforestclassifier__max_depth,None,None,None,15,None,22,10,10,10,5
param_randomforestclassifier__max_features,0.261891,0.37365,0.602432,0.759631,0.5145,0.00474704,0.856175,0.404435,0.307591,0.177278
param_randomforestclassifier__n_estimators,286,109,52,128,52,89,65,137,283,201
param_simpleimputer__strategy,most_frequent,most_frequent,most_frequent,median,most_frequent,median,most_frequent,most_frequent,median,median
params,"{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 15, 'ran...","{'randomforestclassifier__max_depth': None, 'r...","{'randomforestclassifier__max_depth': 22, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 10, 'ran...","{'randomforestclassifier__max_depth': 5, 'rand..."
split0_test_score,0.803333,0.800202,0.798535,0.796667,0.796717,0.793636,0.765606,0.758838,0.757727,0.715758


In [0]:
pipeline = search.best_estimator_
y_pred = pipeline.predict(X_test)

In [0]:
submission = test[['id']].copy()
submission['status_group'] = y_pred
submission.to_csv('VMsubmission.csv', index=False)

In [127]:
# checking best hyperparameters with GridSearchCV
param_grid = {
    'simpleimputer__strategy': ['median', 'most_frequent'], 
}

search = GridSearchCV(
    pipeline, 
    param_grid=param_grid,  
    cv=3, 
    scoring='accuracy', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train);

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done   4 out of   6 | elapsed:  2.3min remaining:  1.1min
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  3.4min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:  3.4min finished


In [129]:
print('Best hyperparameters', search.best_params_)
print('Cross-validation accuracy', search.best_score_)
pd.DataFrame(search.cv_results_).sort_values(by='rank_test_score').T

Best hyperparameters {'simpleimputer__strategy': 'median'}
Cross-validation accuracy 0.8005892255892256


,0,1
mean_fit_time,56.9266,58.9226
std_fit_time,0.286768,0.575658
mean_score_time,2.97522,3.34157
std_score_time,0.126667,0.281891
param_simpleimputer__strategy,median,most_frequent
params,{'simpleimputer__strategy': 'median'},{'simpleimputer__strategy': 'most_frequent'}
split0_test_score,0.802172,0.801869
split1_test_score,0.800202,0.801717
split2_test_score,0.799394,0.797222
mean_test_score,0.800589,0.800269


In [0]:
# Splitting data in a time series manner
val = train[train['date_recorded'] >= '2013-02-01']
train = train[train['date_recorded'] < '2013-02-01']

In [151]:
print(train.shape)
print(val.shape)
print(test.shape)

(41381, 41)
(18019, 41)
(14358, 40)


In [0]:
def data_wrangle(X):
  # using same method for all data

  # create a copy
  X = X.copy()

  # values of latitude near zero will be treated as zero
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  
  # date_recorded appears as object, change into datetime format
  X['date_recorded'] = pd.to_datetime(X['date_recorded'],infer_datetime_format=True)  

  # extract year and month of inspection
  X['month_inspection'] = X['date_recorded'].dt.month
  X['year_inspection'] = X['date_recorded'].dt.year

  # extract number of years till inspection
  X['inspection_waiting_time'] = X['year_inspection'] - X['construction_year']

  # When columns have zeros and shouldn't, they are like null values.
  # So we will replace the zeros with nulls, and impute missing values later.
  cols_to_clean = ['longitude', 'latitude', 'amount_tsh','construction_year',
                   'gps_height', 'permit','public_meeting', 'region_code',
                   'source', 'quantity','quality_group', 'payment',
                   'water_quality', 'source_class','management',
                   'management_group', 'population','installer','funder',
                   'scheme_management', 'scheme_name']
  for col in cols_to_clean:
    X[col] = X[col].replace(0, np.nan)
    X[col] = X[col].replace('unknown', np.nan)
    X[col+'_MISSING'] = X[col].isnull()

  X['inspection_waiting_time_MISSING'] = X['inspection_waiting_time'].isnull()

  # binning numeric values
  cy_cut_points = [0, 0.1, 1980, 2000, 2013]
  cy_label_names = ['0-0.1', '1960-1980', '1981-2000', '2001-2013']
  X['construction_year_cat'] = pd.cut(X['construction_year'], cy_cut_points, labels=cy_label_names)

  gps_cut_points = [-63, 250, 1100, 1600, 2770]
  gps_label_names = ['-63-250', '251-1100', '1101-1600', '1601-2700']
  X['gps_height_cat'] = pd.cut(X['gps_height'], gps_cut_points, labels=gps_label_names)

  # quantity & quantity_group same as payment & payment_type are duplicates
  # drop duplicates, date_recorded, less relevance columns and columns used for binning
  X = X.drop(columns=['quantity_group','payment_type','date_recorded', 'recorded_by',
                      'gps_height','construction_year'])

  # create a top number of values in categorical features with too many unique or little values:
  ET_top6 = X['extraction_type'].value_counts()[:6].index
  ETG_top6 = X['extraction_type_group'].value_counts()[:6].index
  M_top7 = X['management'].value_counts()[:7].index
  S_top5 = X['source'].value_counts()[:5].index
  WQ_top2 = X['water_quality'].value_counts()[:2].index
  W_top4 = X['waterpoint_type'].value_counts()[:4].index


  # Where categorical value is not on the top values,
  # replace the value with 'Other'
  X.loc[~X['extraction_type'].isin(ET_top6), 'extraction_type'] = 'Other'
  X.loc[~X['extraction_type_group'].isin(ETG_top6), 'extraction_type_group'] = 'Other'
  X.loc[~X['management'].isin(M_top7), 'management'] = 'Other'
  X.loc[~X['source'].isin(S_top5), 'source'] = 'Other'
  X.loc[~X['water_quality'].isin(WQ_top2), 'water_quality'] = 'Other'
  X.loc[~X['waterpoint_type'].isin(W_top4), 'waterpoint_type'] = 'Other'

  # return my data_wrangle
  return X

train = data_wrangle(train)
val = data_wrangle(val)
test = data_wrangle(test) 


# defining my y vector
target = 'status_group'

# Get my train DF without the target
train_features = train.drop(columns=[target])

# list of the numeric features
numeric_features = train_features.select_dtypes(include='number').columns.tolist()

# series with the cardinality of the nonnumeric features
cardinality = train_features.select_dtypes(exclude='number').nunique()

# list of all categorical features with cardinality <= 50
categorical_features = cardinality[cardinality <= 50].index.tolist()

# Combine the lists 
features = numeric_features + categorical_features

# arranging my X features and y vector
X_train = train[features]
y_train = train[target]
X_val = val[features]
y_val = val[target]
X_test = test[features]

In [144]:
# Creating my pipeline
pipeline = make_pipeline(
    ce.OrdinalEncoder(), 
    SimpleImputer(strategy='median'), 
    RandomForestClassifier(n_estimators=100, max_depth=24, random_state=42, n_jobs=-1)
)

# Fitting on train, scoring on val
pipeline.fit(X_train, y_train)
print('Validation Accuracy', pipeline.score(X_val, y_val))

Validation Accuracy 0.7151895221710417


In [0]:
y_pred2 = pipeline.predict(X_test)

In [0]:
submission = test[['id']].copy()
submission['status_group'] = y_pred2
submission.to_csv('VMsubmission2.csv', index=False)